# Solver that can play Semantle online

Online guessing with USE semantics and webdriver integration

In [1]:
from english_words import english_words_set
import similarity_model as sm
from gradient_solver import GradientSolver
from cohort_bayes_solver import CohortBayesSolver

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
import time

## Similarity model

In [3]:
base_vocabulary = [w.lower() for w in english_words_set]
similarity_use = sm.SimilarityModelUSE(base_vocabulary)
similarity_w2v = sm.SimilarityModelW2V(base_vocabulary)

2022-07-22 21:15:55.353333: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Incorporate game history

In [4]:
def merge_guesses(solver, guesses, scores):
    unmerged = []
    for i, gw in enumerate(guesses):
        merged = False
        if gw in solver.similarity.vocabulary:
            gi = solver.similarity.word_index(gw)
            if not solver.guess_merged(gi):
                solver.merge_guess(gi, scores[i], 100)
                merged = True
        if not merged:
            unmerged.append(i)
    return unmerged

## Read game and guess

In [5]:
def live_guess(driver, guess):
    guess_input = driver.find_element(By.ID, "guess")
    guess_input.send_keys(guess)
    guess_input.send_keys(Keys.RETURN)

def guess_missed(driver):
    guess_error = driver.find_element(By.ID, "error")
    return guess_error.is_displayed()
    
def clear_guess(driver):
    guess_input = driver.find_element(By.ID, "guess")
    guess_input.clear()

In [6]:
def get_scores(driver):
    guess_table = driver.find_element(By.ID, "guesses")
    rows = guess_table.find_elements(By.TAG_NAME, "tr")
    guesses = []
    scores = []
    for row in rows:
        cols = row.find_elements(By.TAG_NAME, "td")
        if len(cols) >= 3:
            guesses.append(cols[1].text)
            scores.append(float(cols[2].text))
    return guesses, scores

## Init solver and game

In [7]:
solver = GradientSolver(similarity_use, seeds=3, log=True)

In [8]:
driver = webdriver.Firefox()
driver.get("https://semantle.com")
assert "Semantle" in driver.title
try:
    elem = WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.ID, "rules-close")))
finally:
    driver.find_element(By.ID, "rules-close").click()

## Automatically solve

In [9]:
guesses, scores = [], []
while not (len(scores) > 0 and max(scores) > 99.9):
    guess = solver.similarity.word_string(solver.make_guess())
    live_guess(driver, guess)
    time.sleep(1)
    missed = guess_missed(driver)
    if missed: solver.merge_guess(solver.similarity.word_index(guess), 0, 100)
    clear_guess(driver)
    if missed: time.sleep(1)
    try:
        guesses, scores = get_scores(driver)
    except StaleElementReferenceException:
        pass
    finally:
        merge_guesses(solver, guesses, scores)

In [10]:
driver.quit()